## Find Stats

workspace prep

In [1]:
import statsapi
import pandas as pd
import numpy as np
import os
import time
from pybaseball import  playerid_lookup
from pybaseball import  statcast_pitcher
from pybaseball import cache
cache.enable()


# Get the current working directory  
current_dir = os.getcwd() 
program_dir = r'C:\Users\jvgas\OneDrive\Desktop\learning\github\baseballStatistics\py4da\baseballStatistics\programs\programs'
os.chdir(program_dir)

 
print("Current Working Directory:", program_dir)  

data_dir = r'C:\Users\jvgas\OneDrive\Desktop\learning\github\baseballStatistics\py4da\baseballStatistics\datasets\jimbo datasets'
print('')
print("Current Saving Directory:", data_dir)

Current Working Directory: C:\Users\jvgas\OneDrive\Desktop\learning\github\baseballStatistics\py4da\baseballStatistics\programs\programs

Current Saving Directory: C:\Users\jvgas\OneDrive\Desktop\learning\github\baseballStatistics\py4da\baseballStatistics\datasets\jimbo datasets


Importing Files

In [2]:
# Takes the file's folder
filepath = os.path.join(data_dir, 'events2025.csv')
filepath2 = os.path.join(data_dir, 'trimmed retrosheet data/2025/Players2025.csv')

# read the CSV file
events2025 = pd.read_csv(filepath)
events2025 = events2025.drop('Unnamed: 0', axis=1)

players2025 = pd.read_csv(filepath2)
players2025 = players2025.drop('Unnamed: 0', axis=1)

# reading xlsx file
filepath3 = os.path.join(data_dir, '2025 MLB Constants.xlsx')

constants = pd.read_excel(filepath3, sheet_name = 'wOBA & FIP Constants', index_col = 0)
parkFactorDF = pd.read_excel(filepath3, sheet_name = 'Park Factors', index_col = 0)

Statistic Functions

In [123]:
from itertools import count

def parkFactor(playerDF):

    playerDF.loc[playerDF['Inning TopBot'] == 'Bot', 'Player Team'] = playerDF['Home Team']
    playerDF.loc[playerDF['Inning TopBot'] == 'Top', 'Player Team'] = playerDF['Away Team']
    playerTeam = playerDF['Player Team'].iloc[0]
    print(playerName + ' plays for ' + playerTeam)

    parkFactor = (parkFactorDF.loc[parkFactorDF['Abbreviation'] == playerTeam, 'Basic (5yr)']).iloc[0]
    parkFactor = parkFactor.astype(float)
    parkFactor = parkFactor/100
    parkFactorSTR = str(parkFactor)
    print(playerTeam + ' has a park factor of ' + parkFactorSTR)
    return(parkFactor)

def gamesPlayed(playerDF):

    # Count rows where Regular Season Games Played
    count = ((playerDF['Game Type'] == 'R') & ((playerDF['Batter PA This Game'] == 0) & (playerDF['Pitch Number'] == 1))).sum()
    print(f"Games Played: {count}")
    return(count)

def atBats(playerDF):
    count = (((playerDF['Game Type'] == 'R') & ((playerDF['Events'] == 'fielders_choice_out') | (playerDF['Events'] == 'single') | (playerDF['Events'] == 'double') | (playerDF['Events'] == 'triple') | (playerDF['Events'] == 'home_run') | (playerDF['Events'] == 'strikeout') | (playerDF['Events'] == 'field_out') | (playerDF['Events'] == 'grounded_into_double_play') | (playerDF['Events'] == 'force_out') | (playerDF['Events'] == 'double_play') | (playerDF['Events'] == 'field_error')))).sum()
    print(f'At Bats: {count}')
    return(count)

def plateAppearances(playerDF):
    count = ((playerDF['Game Type'] == 'R') & ((playerDF['Events'] == 'walk') | (playerDF['Events'] == 'sac_fly') | (playerDF['Events'] == 'strikeout') | (playerDF['Events'] == 'fielders_choice_out') | (playerDF['Events'] == 'field_error') | (playerDF['Events'] == 'double') | (playerDF['Events'] == 'single') | (playerDF['Events'] == 'field_out') | (playerDF['Events'] == 'home_run') | (playerDF['Events'] == 'triple')| (playerDF['Events'] == 'grounded_into_double_play') | (playerDF['Events'] == 'catcher_interf') | (playerDF['Events'] == 'force_out') | (playerDF['Events'] == 'double_play') | (playerDF['Events'] == 'hit_by_pitch') | (playerDF['Events'] == 'fielders_choice'))).sum()
    count = float(count)
    print(f'Plate Appearances: {count}')
    return(count)

def hits(playerDF):
    count = ((playerDF['Game Type'] == 'R') & ((playerDF['Events'] == 'single') | (playerDF['Events'] == 'double') | (playerDF['Events'] == 'triple') | (playerDF['Events'] == 'home_run'))).sum()
    print(f'Hits: {count}')
    return(count)

def singles(playerDF):
    count = ((playerDF['Game Type'] == 'R') & ((playerDF['Events'] == 'single'))).sum()
    print(f'Singles: {count}')
    return(count)

def doubles(playerDF):
    count = ((playerDF['Game Type'] == 'R') & ((playerDF['Events'] == 'double'))).sum()
    print(f'Doubles: {count}')
    return(count)

def triples(playerDF):
    count = ((playerDF['Game Type'] == 'R') & ((playerDF['Events'] == 'triple'))).sum()
    print(f'Triples: {count}')
    return(count)

def homeRuns(playerDF):
    count = ((playerDF['Game Type'] == 'R') &((playerDF['Events'] == 'home_run'))).sum()
    print(f'Home Runs: {count}')
    return(count)

def runsScored(events2025):
    playerScores = events2025.loc[(events2025['Game Type'] == 'R') & events2025['Play Description'].str.contains(playerName + ' scores.') | events2025['Play Description'].str.contains(playerName + ' homers.')]
    count = len(playerScores)
    print('Runs Scored: ' + str(count))
    return(count)

def runsBattedIn(playerDF):

    playerRBI = playerDF.loc[(playerDF['Game Type'] == 'R') & (playerDF['Batting Team Score'] < playerDF['Post-Pitch Batting Team Score']) & ((playerDF['Events'] != 'error') & (playerDF['Events'] != 'grounded_into_double_play'))]
    playerRBI['Pitch Run Differential'] = playerRBI['Post-Pitch Batting Team Score'] - playerRBI['Batting Team Score']
    RBIs = playerRBI['Pitch Run Differential'].sum()
    count = int(RBIs)
    print('Runs Batted In: ' + str(count))  
    return(count)
  
def walks(playerDF):
    playerWalks = playerDF.loc[(playerDF['Game Type'] == 'R') & ((playerDF['Events'] == 'walk') | (playerDF['Events'] == 'intent_walk'))]
    count = len(playerWalks)
    print('Walks: ' + str(count))
    return(count)

def IBB(playerDF):
    playerIBBs = playerDF.loc[(playerDF['Game Type'] == 'R') & (playerDF['Events'] == 'intent_walk')]
    count = len(playerIBBs)
    print('Intentional Walks: ' + str(count))
    return(count)

def strikeouts(playerDF):
    playerStrikeouts = playerDF.loc[(playerDF['Game Type'] == 'R') & playerDF['Play Description'].str.contains(playerName + ' strikes out') | playerDF['Play Description'].str.contains(playerName + ' called out on strikes')]
    count = len(playerStrikeouts)
    print('Strikeouts: ' + str(count))
    return(count)

def strikeoutsLooking(playerDF):
    playerKlooking = playerDF.loc[(playerDF['Game Type'] == 'R') & playerDF['Play Description'].str.contains(playerName + ' called out on strikes')]
    count = len(playerKlooking)
    print('Strikeouts Looking: ' + str(count))
    return(count)

def HBP(playerDF):
    playerHBP = playerDF.loc[(playerDF['Game Type'] == 'R') & ((playerDF['Pitch Description'] == 'hit_by_pitch'))]
    count = len(playerHBP)
    print('Hit by Pitch: ' + str(count))
    return(count)

def sacFlies(playerDF):
    playerSacFly = playerDF.loc[(playerDF['Game Type'] == 'R') & ((playerDF['Play Description'].str.contains('sacrifice fly')))]
    count = len(playerSacFly)
    print('Sacrifice Flies: ' + str(count))
    return(count)

def sacBunts(playerDF):
    playerSacBunt = playerDF.loc[(playerDF['Game Type'] == 'R') & ((playerDF['Events'] == 'sac_bunt'))]
    count = len(playerSacBunt)
    print('Sacrifice Bunts: ' + str(count))
    return(count)

def GIDP(playerDF):
    playerGIDP = playerDF.loc[(playerDF['Game Type'] == 'R') & ((playerDF['Events'] == 'grounded_into_double_play'))]
    count = len(playerGIDP)
    print('Times', playerName,  'grounded into a double play: ' + str(count))
    return(count)

def steals(playerDF):
    playerSteals = playerDF.loc[(playerDF['Game Type'] == 'R') & ((playerDF['Play Description'].str.contains('steals')))]
    count = len(playerSteals)
    print('Number of bases stolen: ' + str(count))
    return(count)

def caughtStealing(playerDF):
    playerCaughtStealing = playerDF.loc[(playerDF['Game Type'] == 'R') & ((playerDF['Play Description'].str.contains('caught stealing')))]
    count = len(playerCaughtStealing)
    print('Number of times caught stealing: ' + str(count))
    return(count)

def avg(playerDF):
    playerAVG = (round(hits(playerDF) / atBats(playerDF), 3))
    print('Batting Average: ' + str(playerAVG))
    return(playerAVG)

def walkRate(playerDF):
    walkRate = round(walks(playerDF) / plateAppearances(playerDF), 4)
    walkRate = walkRate * 100
    print(playerName + "'s Walk Rate: " + str(walkRate) + "%")
    return(walkPercentage)

def KRate(playerDF):
    KPercentage = round(strikeouts(playerDF) / plateAppearances(playerDF), 4)
    KPercentage = (KPercentage) * 100
    print(playerName + "'s Strikeout Rate: " + str(KPercentage) + "%")
    return(KPercentage)

def WalktoKRatio(playerDF):
    BBtoKRatio = round(walks(playerDF) / strikeouts(playerDF), 3)
    print(playerName + "'s BB/K Ratio: " + str(BBtoKRatio))
    return(BBtoKRatio)

def OnBase(playerDF):
    OBP = (((hits(playerDF) + walks(playerDF) + HBP(playerDF)) / (atBats(playerDF) + walks(playerDF) + HBP(playerDF) + sacFlies(playerDF))))
    OBP  = round(OBP, 3)
    print(playerName + "'s On-Base Percentage: " + str(OBP))
    return(OBP)

def slugging(playerDF):
    TB = singles(playerDF) + (2 * doubles(playerDF)) + (3 * triples(playerDF)) + (4 * homeRuns(playerDF))
    SLG = round(TB / atBats(playerDF), 3)
    print(playerName + "'s Slugging Percentage: " + str(SLG))
    return(SLG)

def OPS(playerDF):
    OnBasePlusSlg = OnBase(playerDF) + slugging(playerDF)
    OnBasePlusSlg = round(OnBasePlusSlg, 3)
    print(playerName + "'s OPS: " + str(OnBasePlusSlg))
    return(OnBasePlusSlg)

def ISO(playerDF):
    isolatedPower = slugging(playerDF) - avg(playerDF)
    isolatedPower = round(isolatedPower, 3)
    print(playerName + "'s Isolated Power: " + str(isolatedPower))
    return(isolatedPower)

def BABIP(playerDF):
    battingAvgOnBallsInPlay = ((hits(playerDF) - homeRuns(playerDF)) / (atBats(playerDF) - strikeouts(playerDF) - homeRuns(playerDF) + sacFlies(playerDF) + sacBunts(playerDF)))
    battingAvgOnBallsInPlay = round(battingAvgOnBallsInPlay, 3)
    print(playerName + "'s BABIP: " + str(battingAvgOnBallsInPlay))
    return(battingAvgOnBallsInPlay)

def wSB(playerDF):
    weightedSB = (steals(playerDF) * 0.200) + (caughtStealing(playerDF) * (-0.410)) - ((((steals(playerDF) * 200) + (caughtStealing(playerDF) * (-0.410)))/(hits(playerDF) + walks(playerDF) + HBP(playerDF) - IBB(playerDF))) * (hits(playerDF) + walks(playerDF) + HBP(playerDF) - IBB(playerDF)))
    weightedSB = round(weightedSB, 3)
    print(playerName + "'s wSB: " + str(weightedSB))
    return(weightedSB)

def wOBA(playerDF):
    weightedOnBaseAverage = (((0.691 * (walks(playerDF) - IBB(playerDF))) + (0.722 * HBP(playerDF)) + (0.882 * singles(playerDF)) +(1.252 * doubles(playerDF)) + (1.584 * triples(playerDF)) + (2.037 * homeRuns(playerDF)))
                          / (atBats(playerDF) + (walks(playerDF) - IBB(playerDF)) + sacFlies(playerDF) + HBP(playerDF)))
    weightedOnBaseAverage = round(weightedOnBaseAverage, 3)
    print(playerName + "'s wOBA: " + str(weightedOnBaseAverage))
    return(weightedOnBaseAverage)

def wRC(playerDF):
    weightedRunsCreated = ((((wOBA(playerDF) - 0.313) / 1.232) + 0.118) * plateAppearances(playerDF))
    weightedRunsCreated = round(weightedRunsCreated, 0)
    print(playerName + "'s wRC: " + str(weightedRunsCreated))
    return(weightedRunsCreated)

def wRAA(playerDF):
    weightedRunsAboveAverage = (((wOBA(playerDF) - 0.313) / 1.232) * plateAppearances(playerDF))
    weightedRunsAboveAverage = round(weightedRunsAboveAverage, 0)
    print(playerName + "'s wRAA: " + str(weightedRunsAboveAverage))
    return(weightedRunsAboveAverage)

def wRCPlus(playerDF):
    playerLeague = parkFactorDF.loc[parkFactorDF['Abbreviation'] == playerTeam, 'League'].iloc[0]
    playerLeague = str(playerLeague)

    if playerLeague == 'AL':
        leaguewRC = (constants['AL wRC'].loc[constants.index[0]])
        leaguewRC = leaguewRC.astype(int)

        leaguePA = (constants['AL PA'].loc[constants.index[0]])
        leaguePA = leaguePA.astype(int)
        print(leaguewRC, leaguePA)
    else:
        leaguewRC = (constants['NL wRC'].loc[constants.index[0]])
        leaguewRC = leaguewRC.astype(int)

        leaguePA = (constants['NL PA'].loc[constants.index[0]])
        leaguePA = leaguePA.astype(int)
        print(leaguewRC, leaguePA)
     
    weightedRunsCreatedPlus = (((wRAA(playerDF)/plateAppearances(playerDF) + 0.118) + (0.118 - parkFactor(playerDF) * 0.118)) / (leaguewRC/leaguePA)) * 100
    weightedRunsCreatedPlus = round(weightedRunsCreatedPlus, 0)
    print(weightedRunsCreatedPlus)
    return(weightedRunsCreatedPlus)

def GBoverFB(playerDF):
    groundBallCount = len(playerDF.loc[playerDF['Batted Ball Type'] == 'ground_ball'])
    flyBallCount = len(playerDF.loc[playerDF['Batted Ball Type'] == 'fly_ball'])
    GBFBRatio = round(groundBallCount / flyBallCount, 2)
    print(GBFBRatio)
    return(GBFBRatio)

def ldRate(playerDF):
    linerCount = ((playerDF['Game Type'] == 'R') & ((playerDF['Batted Ball Type']) == 'line_drive')).sum()
    linerRate = round((linerCount / (atBats(playerDF) - strikeouts(playerDF) - homeRuns(playerDF) + sacFlies(playerDF) + sacBunts(playerDF))) * 100, 2)
    print(linerRate)
    return(linerRate)

def gbRate(playerDF):
    grounderCount = ((playerDF['Game Type'] == 'R') & ((playerDF['Batted Ball Type']) == 'ground_ball')).sum()
    grounderRate = round((grounderCount / (atBats(playerDF) - strikeouts(playerDF) - homeRuns(playerDF) + sacFlies(playerDF) + sacBunts(playerDF))) * 100, 2)
    print(grounderRate)
    return(grounderRate)

def fbRate(playerDF):
    flyCount = ((playerDF['Game Type'] == 'R') & ((playerDF['Batted Ball Type']) == 'fly_ball')).sum()
    flyRate = round((flyCount / (atBats(playerDF) - strikeouts(playerDF) - homeRuns(playerDF) + sacFlies(playerDF) + sacBunts(playerDF))) * 100, 2)
    print(flyRate)
    return(flyRate)

def IFFB(playerDF):
    flyCount = ((playerDF['Game Type'] == 'R') & ((playerDF['Batted Ball Type']) == 'fly_ball')).sum()
    infieldFlyCount = ((playerDF['Game Type'] == 'R') & ((playerDF['Batted Ball Type']) == 'popup')).sum()

    infieldFlyRate = round((infieldFlyCount / flyCountF) * 100, 2)
    print(infieldFlyRate)
    return(infieldFlyRate)

IFFB%

In [ ]:
flyCount = ((playerDF['Game Type'] == 'R') & ((playerDF['Batted Ball Type']) == 'fly_ball')).sum()
infieldFlyCount = ((playerDF['Game Type'] == 'R') & ((playerDF['Batted Ball Type']) == 'popup')).sum()

infieldFlyRate = round((infieldFlyCount / flyCountF) * 100, 2)
print(infieldFlyRate)
return(infieldFlyRate)

Saving playerDF

In [105]:
# Change CWD  
os.chdir(data_dir)  
 
# Verify the new CWD  
playerDF.to_excel('playerDF.xlsx')
print('File saved successfully in the specified directory:', data_dir)

File saved successfully in the specified directory: C:\Users\jvgas\OneDrive\Desktop\learning\github\baseballStatistics\py4da\baseballStatistics\datasets\jimbo datasets


GB/FB

In [111]:
groundBallCount = len(playerDF.loc[playerDF['Batted Ball Type'] == 'ground_ball'])
flyBallCount = len(playerDF.loc[playerDF['Batted Ball Type'] == 'fly_ball'])
GBFBRatio = round(groundBallCount / flyBallCount, 2)

print(GBFBRatio)

print(str(groundBallCount) + ' ground balls and ' + str(flyBallCount) + ' fly balls')

1.34
158 ground balls and 118 fly balls


LD%

In [116]:
linerCount = ((playerDF['Game Type'] == 'R') & ((playerDF['Batted Ball Type']) == 'line_drive')).sum()
linerRate = round((linerCount / (atBats(playerDF) - strikeouts(playerDF) - homeRuns(playerDF) + sacFlies(playerDF) + sacBunts(playerDF))) * 100, 2)
print(linerRate)

At Bats: 464
Strikeouts: 99
Home Runs: 26
Sacrifice Flies: 2
Sacrifice Bunts: 0
26.69


GB%

In [ ]:
grounderCount = ((playerDF['Game Type'] == 'R') & ((playerDF['Batted Ball Type']) == 'ground_ball')).sum()
grounderRate = round((grounderCount / (atBats(playerDF) - strikeouts(playerDF) - homeRuns(playerDF) + sacFlies(playerDF) + sacBunts(playerDF))) * 100, 2)
print(grounderRate)

At Bats: 464
Strikeouts: 99
Home Runs: 26
Sacrifice Flies: 2
Sacrifice Bunts: 0
43.11


FB%

In [120]:
flyCount = ((playerDF['Game Type'] == 'R') & ((playerDF['Batted Ball Type']) == 'fly_ball')).sum()
flyRate = round((flyCount / (atBats(playerDF) - strikeouts(playerDF) - homeRuns(playerDF) + sacFlies(playerDF) + sacBunts(playerDF))) * 100, 2)
print(flyRate)

At Bats: 464
Strikeouts: 99
Home Runs: 26
Sacrifice Flies: 2
Sacrifice Bunts: 0
30.79


parkFactor

In [ ]:
playerDF.loc[playerDF['Inning TopBot'] == 'Bot', 'Player Team'] = playerDF['Home Team']
playerDF.loc[playerDF['Inning TopBot'] == 'Top', 'Player Team'] = playerDF['Away Team']
playerTeam = playerDF['Player Team'].iloc[0]
print(playerName + ' plays for ' + playerTeam)

parkFactor = (parkFactorDF.loc[parkFactorDF['Abbreviation'] == playerTeam, 'Basic (5yr)']).iloc[0]
parkFactor = parkFactor.astype(float)
parkFactor = parkFactor/100


Ben Rice plays for NYY


In [69]:
parkFactor

0.96

wRC+

In [ ]:
playerLeague = parkFactorDF.loc[parkFactorDF['Abbreviation'] == playerTeam, 'League'].iloc[0]
playerLeague = str(playerLeague)

if playerLeague == 'AL':
    leaguewRC = (constants['AL wRC'].loc[constants.index[0]])
    leaguewRC = leaguewRC.astype(int)

    leaguePA = (constants['AL PA'].loc[constants.index[0]])
    leaguePA = leaguePA.astype(int)
    print(leaguewRC, leaguePA)
else:
    leaguewRC = (constants['NL wRC'].loc[constants.index[0]])
    leaguewRC = leaguewRC.astype(int)

    leaguePA = (constants['NL PA'].loc[constants.index[0]])
    leaguePA = leaguePA.astype(int)
    print(leaguewRC, leaguePA)
     
weightedRunsCreatedPlus = (((wRAA(playerDF)/plateAppearances(playerDF) + 0.118) + (0.118 - parkFactor(playerDF) * 0.118)) / (leaguewRC/leaguePA)) * 100
weightedRunsCreatedPlus = round(weightedRunsCreatedPlus, 0)
print(weightedRunsCreatedPlus)

In [87]:
wRCPlus(playerDF)

10750 91255
Walks: 50
Intentional Walks: 0
Hit by Pitch: 9
Singles: 61
Doubles: 28
Triples: 4
Home Runs: 26
At Bats: 464
Walks: 50
Intentional Walks: 0
Sacrifice Flies: 2
Hit by Pitch: 9
Ben Rice's wOBA: 0.36
Plate Appearances: 528.0
Ben Rice's wRAA: 20.0
Plate Appearances: 528.0
Ben Rice plays for NYY
NYY has a park factor of 0.99
133.0


In [79]:
print(type(parkFactor(playerDF)))


Ben Rice plays for NYY
NYY has a park factor of 0.96
<class 'numpy.float64'>


wRAA

In [21]:
weightedRunsAboveAverage = (((wOBA(playerDF) - 0.313) / 1.232) * plateAppearances(playerDF))
weightedRunsAboveAverage = round(weightedRunsAboveAverage, 0)
print(playerName + "'s wRAA: " + str(weightedRunsAboveAverage))
#return(weightedRunsAboveAverage)

Walks: 50
Intentional Walks: 0
Hit by Pitch: 9
Singles: 61
Doubles: 28
Triples: 4
Home Runs: 26
At Bats: 464
Walks: 50
Intentional Walks: 0
Sacrifice Flies: 2
Hit by Pitch: 9
Ben Rice's wOBA: 0.36
Plate Appearances: 528
Ben Rice's wRAA: 20.0


In [22]:
wRAA(playerDF)

Walks: 50
Intentional Walks: 0
Hit by Pitch: 9
Singles: 61
Doubles: 28
Triples: 4
Home Runs: 26
At Bats: 464
Walks: 50
Intentional Walks: 0
Sacrifice Flies: 2
Hit by Pitch: 9
Ben Rice's wOBA: 0.36
Plate Appearances: 528
Ben Rice's wRAA: 20.0


20.0

wRC

In [ ]:
weightedRunsCreated = ((((wOBA(playerDF) - 0.313) / 1.232) + 0.118) * plateAppearances(playerDF))
weightedRunsCreated = round(weightedRunsCreated, 0)
print(playerName + "'s wRC: " + str(weightedRunsCreated))
#return(weightedRunsCreated)

Walks: 50
Intentional Walks: 0
Hit by Pitch: 9
Singles: 61
Doubles: 28
Triples: 4
Home Runs: 26
At Bats: 464
Walks: 50
Intentional Walks: 0
Sacrifice Flies: 2
Hit by Pitch: 9
Ben Rice's wOBA: 0.36
Plate Appearances: 528
Ben Rice's wRC: 82.0


In [ ]:
wRC(playerDF)

Walks: 50
Intentional Walks: 0
Hit by Pitch: 9
Singles: 61
Doubles: 28
Triples: 4
Home Runs: 26
At Bats: 464
Walks: 50
Intentional Walks: 0
Sacrifice Flies: 2
Hit by Pitch: 9
Ben Rice's wOBA: 0.36
Plate Appearances: 528
Ben Rice's wRC: 82.0


82.0

wOBA

In [ ]:
weightedOnBaseAverage = (((0.691 * (walks(playerDF) - IBB(playerDF))) + (0.722 * HBP(playerDF)) + (0.882 * singles(playerDF)) +(1.252 * doubles(playerDF)) + (1.584 * triples(playerDF)) + (2.037 * homeRuns(playerDF)))
                          / (atBats(playerDF) + (walks(playerDF) - IBB(playerDF)) + sacFlies(playerDF) + HBP(playerDF)))
weightedOnBaseAverage = round(weightedOnBaseAverage, 3)
print(playerName + "'s wOBA: " + str(weightedOnBaseAverage))
#return(weightedOnBaseAverage)

Walks: 50
Intentional Walks: 0
Hit by Pitch: 9
Singles: 61
Doubles: 28
Triples: 4
Home Runs: 26
At Bats: 464
Walks: 50
Intentional Walks: 0
Sacrifice Flies: 2
Hit by Pitch: 9
Ben Rice's wOBA: 0.36


In [ ]:
wOBA(playerDF)

Walks: 50
Intentional Walks: 0
Hit by Pitch: 9
Singles: 61
Doubles: 28
Triples: 4
Home Runs: 26
At Bats: 464
Walks: 50
Intentional Walks: 0
Sacrifice Flies: 2
Hit by Pitch: 9
Ben Rice's wOBA: 0.36


0.36

weighted Stolen Bases  

In [ ]:
weightedSB = (steals(playerDF) * 0.200) + (caughtStealing(playerDF) * (-0.410)) - ((((steals(playerDF) * 200) + (caughtStealing(playerDF) * (-0.410)))/(hits(playerDF) + walks(playerDF) + HBP(playerDF) - IBB(playerDF))) * (hits(playerDF) + walks(playerDF) + HBP(playerDF) - IBB(playerDF)))
weightedSB = round(weightedSB, 3)
print(playerName + "'s wSB: " + str(weightedSB))
return(weightedSB)

Number of bases stolen: 0
Number of times caught stealing: 1
Number of bases stolen: 0
Number of times caught stealing: 1
Hits: 119
Walks: 50
Hit by Pitch: 9
Intentional Walks: 0
Hits: 119
Walks: 50
Hit by Pitch: 9
Intentional Walks: 0
Ben Rice's wSB: 0.0


In [ ]:
wSB(playerDF)

Number of bases stolen: 0
Number of times caught stealing: 1
Number of bases stolen: 0
Number of times caught stealing: 1
Hits: 119
Walks: 50
Hit by Pitch: 9
Intentional Walks: 0
Hits: 119
Walks: 50
Hit by Pitch: 9
Intentional Walks: 0
Ben Rice's wSB: 0.0


0.0

BABIP

In [ ]:
battingAvgOnBallsInPlay = ((hits(playerDF) - homeRuns(playerDF)) / (atBats(playerDF) - strikeouts(playerDF) - homeRuns(playerDF) + sacFlies(playerDF)))
battingAvgOnBallsInPlay = round(battingAvgOnBallsInPlay, 3)
print(playerName + "'s BABIP: " + str(battingAvgOnBallsInPlay))
return(battingAvgOnBallsInPlay)

Hits: 119
Home Runs: 26
At Bats: 464
Strikeouts: 99
Home Runs: 26
Sacrifice Flies: 2
Ben Rice's BABIP: 0.273


SyntaxError: 'return' outside function (Temp/ipykernel_5032/2875569526.py, line 4)

In [ ]:
BABIP(playerDF)

Hits: 119
Home Runs: 26
At Bats: 464
Strikeouts: 99
Home Runs: 26
Sacrifice Flies: 2
Ben Rice's BABIP: 0.273


0.273

ISO

In [ ]:
isolatedPower = slugging(playerDF) - avg(playerDF)
isolatedPower = round(isolatedPower, 3)
print(playerName + "'s Isolated Power: " + str(isolatedPower))
#return(isolatedPower)

Singles: 61
Doubles: 28
Triples: 4
Home Runs: 26
At Bats: 464
Ben Rice's Slugging Percentage: 0.502
Hits: 119
At Bats: 464
Batting Average: 0.256
Ben Rice's Isolated Power: 0.246


In [ ]:
ISO(playerDF)

Singles: 61
Doubles: 28
Triples: 4
Home Runs: 26
At Bats: 464
Ben Rice's Slugging Percentage: 0.502
Hits: 119
At Bats: 464
Batting Average: 0.256
Ben Rice's Isolated Power: 0.246


0.246

OPS

In [ ]:
OnBasePlusSlg = OnBase(playerDF) + slugging(playerDF)
OnBasePlusSlg = round(OnBasePlusSlg, 3)
print(playerName + "'s OPS: " + str(OnBasePlusSlg))
#return(OPS)

Hits: 119
Walks: 50
Hit by Pitch: 9
At Bats: 464
Walks: 50
Hit by Pitch: 9
Sacrifice Flies: 2
Ben Rice's On-Base Percentage: 0.339
Singles: 61
Doubles: 28
Triples: 4
Home Runs: 26
At Bats: 464
Ben Rice's Slugging Percentage: 0.502
Ben Rice's OPS: 0.841


In [ ]:
OPS(playerDF)

Hits: 119
Walks: 50
Hit by Pitch: 9
At Bats: 464
Walks: 50
Hit by Pitch: 9
Sacrifice Flies: 2
Ben Rice's On-Base Percentage: 0.339
Singles: 61
Doubles: 28
Triples: 4
Home Runs: 26
At Bats: 464
Ben Rice's Slugging Percentage: 0.502
Ben Rice's OPS: 0.841


0.841

Slugging

In [ ]:
TB = singles(playerDF) + (2 * doubles(playerDF)) + (3 * triples(playerDF)) + (4 * homeRuns(playerDF))
SLG = round(TB / atBats(playerDF), 3)
print(playerName + "'s Slugging Percentage: " + str(SLG))
return(SLG)

In [ ]:
slugging(playerDF)

Singles: 61
Doubles: 28
Triples: 4
Home Runs: 26
At Bats: 464
Ben Rice's Slugging Percentage: 0.502


0.502

walks

In [ ]:
playerWalks = playerDF.loc[(playerDF['Game Type'] == 'R') & ((playerDF['Events'] == 'walk') | (playerDF['Events'] == 'intent_walk'))]
count = len(playerWalks)
print('Walks: ' + str(count))
#return(count)

In [ ]:
walks(playerDF)

IBB

In [ ]:
playerIBB = playerDF.loc[(playerDF['Game Type'] == 'R') & (playerDF['Events'] == 'intent_walk')]
count = len(playerIBB)
print('Intentional Walks: ' + str(count))
#return(count)

In [ ]:
IBB(playerDF)

Strikeouts

In [ ]:
playerStrikeouts = playerDF.loc[(playerDF['Game Type'] == 'R') & playerDF['Play Description'].str.contains(playerName + ' strikes out') | playerDF['Play Description'].str.contains(playerName + ' called out on strikes')]
count = len(playerStrikeouts)
print('Strikeouts: ' + str(count))
return(count)

Strikeouts: 99


In [ ]:
strikeouts(playerDF)

Strikeouts: 99


99

Strikeouts Looking

In [ ]:
playerKlooking = playerDF.loc[(playerDF['Game Type'] == 'R') & playerDF['Play Description'].str.contains(playerName + ' called out on strikes')]
count = len(playerKlooking)
print('Strikeouts Looking: ' + str(count))
#return(count)

Strikeouts Looking: 35


In [ ]:
strikeoutsLooking(playerDF)

Strikeouts Looking: 35


35

Hit-By-Pitch

In [ ]:
playerHBP = playerDF.loc[(playerDF['Game Type'] == 'R') & ((playerDF['Pitch Description'] == 'hit_by_pitch'))]
count = len(playerHBP)
print('Hit by Pitch: ' + str(count))
#return(count)

Hit by Pitch: 9


In [ ]:
HBP(playerDF)

Hit by Pitch: 9


9

Sacrifice Fly

In [ ]:
playerSacFly = playerDF.loc[(playerDF['Game Type'] == 'R') & ((playerDF['Play Description'].str.contains('sacrifice fly')))]
count = len(playerSacFly)
print('Sacrifice Flies: ' + str(count))
#return(count)

Sacrifice Flies: 2


In [ ]:
sacFlies(playerDF)

Sacrifice Flies: 2


2

Sacrifice Bunts

In [ ]:
playerSacBunt = playerDF.loc[(playerDF['Game Type'] == 'R') & ((playerDF['Events'] == 'sac_bunt'))]
count = len(playerSacBunt)
print('Sacrifice Bunts: ' + str(count))
#return(count)

Sacrifice Bunts: 0


In [ ]:
sacBunts(playerDF)

Sacrifice Bunts: 0


0

Ground Into Double Play

In [ ]:
playerGIDP = playerDF.loc[(playerDF['Game Type'] == 'R') & ((playerDF['Events'] == 'grounded_into_double_play'))]
count = len(playerGIDP)
print('Times', playerName,  'grounded into a double play: ' + str(count))
#return(count)

Times Ben Rice grounded into a double play: 18


Steals

In [ ]:
playerSteals = playerDF.loc[(playerDF['Game Type'] == 'R') & ((playerDF['Play Description'].str.contains('steals')))]
count = len(playerSteals)
print('Number of bases stolen: ' + str(count))
#return(count)

Number of bases stolen: 0


In [ ]:
steals(playerDF)

Number of bases stolen: 0


0

Caught Stealing

In [ ]:
playerCaughtStealing = playerDF.loc[(playerDF['Game Type'] == 'R') & ((playerDF['Play Description'].str.contains('caught stealing')))]
count = len(playerCaughtStealing)
print('Number of times caught stealing: ' + str(count))
#return(count)

Number of times caught stealing: 1


In [ ]:
caughtStealing(playerDF)

Number of times caught stealing: 1


1

rbi

In [ ]:
 playerRBI = playerDF.loc[(playerDF['Batting Team Score'] < playerDF['Post-Pitch Batting Team Score']) & (playerDF['Game Type'] == 'R') & (playerDF['Events'] != 'error') & (playerDF['Events'] != 'grounded_into_double_play')]
 playerRBI['Pitch Run Differential'] = playerRBI['Post-Pitch Batting Team Score'] - playerRBI['Batting Team Score']
 RBIs = playerRBI['Pitch Run Differential'].sum()
 count = int(RBIs)
 print('Runs Batted In: ' + str(count))

In [ ]:
runsBattedIn(playerDF)

walks

In [ ]:
playerWalks = playerDF.loc[(playerDF['Game Type'] == 'R') & (playerDF['Events'] == 'walk') | (playerDF['Events'] == 'intent_walk')]
count = len(playerWalks)
print('Walks: ' + str(count))

In [ ]:
walks(playerDF)

Walks: 50


50

runs scored

In [ ]:
playerScores = events2025.loc[(events2025['Game Type'] == 'R') & events2025['Play Description'].str.contains(playerName + ' scores.') | events2025['Play Description'].str.contains(playerName + ' homers.')]
count = len(playerScores)
print('Runs Scored: ' + str(count))
#return(count)

Runs Scored: 74


In [ ]:
runsScored(events2025)

Runs Scored: 74


74

Batting Average

In [ ]:
playerAVG = (round(hits(playerDF) / atBats(playerDF), 3))
print('Batting Average: ' + str(playerAVG))
#return(playerAVG)

Hits: 119
At Bats: 464
Batting Average: 0.256


In [ ]:
avg(playerDF)

Hits: 119
At Bats: 464
Batting Average: 0.256


0.256

BB%

In [ ]:
walkPercentage = round(walks(playerDF) / plateAppearances(playerDF), 4)
walkPercentage = (walkPercentage) * 100
print(playerName + "'s Walk Rate: " + str(walkPercentage) + "%")
#return(walkPercentage)

Walks: 50
Plate Appearances: 528
Ben Rice's Walk Rate: 9.47%


In [ ]:
walkRate(playerDF)

Walks: 50
Plate Appearances: 528
Ben Rice's Walk Rate: 9.47%


9.47

K%

In [ ]:
KPercentage = round(strikeouts(playerDF) / plateAppearances(playerDF), 4)
KPercentage = (KPercentage) * 100
print(playerName + "'s Strikeout Rate: " + str(KPercentage) + "%")
#return(KPercentage)

Strikeouts: 99
Plate Appearances: 528
Ben Rice's Strikeout Rate: 18.75%


In [ ]:
KRate(playerDF)

Strikeouts: 99
Plate Appearances: 528
Ben Rice's Strikeout Rate: 18.75%


18.75

BB/K%

In [ ]:
BBtoKRatio = round(walks(playerDF) / strikeouts(playerDF), 3)
print(playerName + "'s BB/K Ratio: " + str(BBtoKRatio))
#return(BBtoKRatio)

Walks: 50
Strikeouts: 99
Ben Rice's BB/K Ratio: 0.505


In [ ]:
WalktoKRatio(playerDF)

Walks: 50
Strikeouts: 99
Ben Rice's BB/K Ratio: 0.505


0.505

On Base Percentage

In [ ]:
OBP = (((hits(playerDF) + walks(playerDF) + HBP(playerDF)) / (atBats(playerDF) + walks(playerDF) + HBP(playerDF) + sacFlies(playerDF))))
OBP  = round(OBP, 3)
print(playerName + "'s On-Base Percentage: " + str(OBP))

Hits: 119
Walks: 50
Hit by Pitch: 9
At Bats: 464
Walks: 50
Hit by Pitch: 9
Sacrifice Flies: 2
Ben Rice's On-Base Percentage: 0.339


In [ ]:
OnBase(playerDF)

Hits: 119
Walks: 50
Hit by Pitch: 9
At Bats: 464
Walks: 50
Hit by Pitch: 9
Sacrifice Flies: 2
Ben Rice's On-Base Percentage: 0.339


0.339

## Player Lookem Up


In [ ]:
avg(playerDF)

NameError: name 'avg' is not defined

In [6]:
playerName = 'Ben Rice' #Hardcoded for testing
playerFirstName = 'Ben'
playerLastName = 'Rice'



def findStat(playerName): # statType, timeFrame):
    print("Searching for", playerName)
    
    # Using Player Look Up to find PLayer ID
    playerLookup = playerid_lookup(playerLastName, playerFirstName)
    playerID = playerLookup['key_mlbam'][0]
    print(playerID) #Debugging line to ensure correct playerID is found
    

    # Find playerDF
    playerDF = events2025.loc[events2025['Batter ID'] == playerID] 
    print(playerDF.head()) #Debugging line to ensure correct playerDF is found





findStat(playerName)
#findStat(statType = 'Hits', playerName = 'Ben Rice', timeFrame = '07/01/2025 to 08/01/2025)')

Searching for Ben Rice
700250
     Pitch Type   Game Date  Release Speed  X Release  Z Release  \
6946         FS  2025-10-08           89.8      -2.55       5.68   
6947         FF  2025-10-08           94.8      -2.34       5.81   
6948         FF  2025-10-08           96.9      -2.46       5.81   
6949         FS  2025-10-08           87.8      -2.56       5.77   
6950         FS  2025-10-08           90.5      -2.43       5.62   

        Player Name  Batter ID  Pitcher ID Events Pitch Description  ...  \
6946  Hoffman, Jeff     700250      656546   walk              ball  ...   
6947  Hoffman, Jeff     700250      656546    NaN              foul  ...   
6948  Hoffman, Jeff     700250      656546    NaN   swinging_strike  ...   
6949  Hoffman, Jeff     700250      656546    NaN              ball  ...   
6950  Hoffman, Jeff     700250      656546    NaN      blocked_ball  ...   

      Batter Days Until Next Appearance  Drop  X Break (Arm)  \
6946                                NaN 

In [8]:
playerName = 'Ben Rice' #Hardcoded for testing
playerFirstName = 'Ben'
playerLastName = 'Rice'

# Using Player Look Up to find PLayer ID
playerLookup = playerid_lookup(playerLastName, playerFirstName)
playerID = playerLookup['key_mlbam'][0]
print(playerID) #Debugging line to ensure correct playerID is found
    

# Find playerDF
playerDF = events2025.loc[events2025['Batter ID'] == playerID] 
print(playerDF.head()) #Debugging line to ensure correct playerDF is found

700250
     Pitch Type   Game Date  Release Speed  X Release  Z Release  \
6946         FS  2025-10-08           89.8      -2.55       5.68   
6947         FF  2025-10-08           94.8      -2.34       5.81   
6948         FF  2025-10-08           96.9      -2.46       5.81   
6949         FS  2025-10-08           87.8      -2.56       5.77   
6950         FS  2025-10-08           90.5      -2.43       5.62   

        Player Name  Batter ID  Pitcher ID Events Pitch Description  ...  \
6946  Hoffman, Jeff     700250      656546   walk              ball  ...   
6947  Hoffman, Jeff     700250      656546    NaN              foul  ...   
6948  Hoffman, Jeff     700250      656546    NaN   swinging_strike  ...   
6949  Hoffman, Jeff     700250      656546    NaN              ball  ...   
6950  Hoffman, Jeff     700250      656546    NaN      blocked_ball  ...   

      Batter Days Until Next Appearance  Drop  X Break (Arm)  \
6946                                NaN  2.46           1.30   

In [ ]:
statType = 'Games Played'
statValue = len(ilocplayerDF['Batter PA This Game'].value_counts().get(0,0)
print('The value of', statType, 'is', statValue)



In [ ]:
playerDF

In [ ]:
playerName = 'Ben Rice' #Hardcoded for testing
playerFirstName = 'Ben'
playerLastName = 'Rice'

# DO NOT DELETE

Find the PLayer DF from batting2025 using the player ID found in players2025

In [ ]:
playerName = 'Ben Rice'

man = players2025.loc[players2025['Full Name'].str.contains('Ben Rice')]
playerID = man['RetroID'].values[0]
print(playerID)

playerDF = batting2025.loc[batting2025['Player ID'] == playerID]

In [ ]:
def findStat(playerName): # statType, timeFrame):
    print("Searching for", playerName)
    
    # Filter batting2025 dataframe for playerID
    man = players2025.loc[players2025['Full Name']].str.contains(playerName)
    playerID = man['RetroID'].value[0]
    print(playerID) #Debugging line to ensure correct playerID is found

    # Find playerDF
    playerDF = batting.loc[batting2025['Player ID'] == playerID] 
    print(playerDF.head(5)) #Debugging line to ensure correct playerDF is found

In [ ]:
from pybaseball import  playerid_lookup
from pybaseball import  statcast_pitcher


playerLookup = playerid_lookup(playerLastName, playerFirstName)
playerID = playerLookup['key_mlbam'][0]

In [ ]:
playerID

Finding how stat is recorded

In [ ]:
testEvents = (events2025[events2025['Play Description'].str.contains('caught stealing')])

In [ ]:
testEvents

,Pitch Type,Game Date,Release Speed,X Release,Z Release,Player Name,Batter ID,Pitcher ID,Events,Pitch Description,...,Batter Days Until Next Appearance,Drop,X Break (Arm),X Break (Batter),Arm Angle,attack_angle,Attack Direction,Swing Path Tilt,X Position at Intercept,Y Position at Intercept
339,FF,2025-11-01,99.6,-2.27,5.51,"Ohtani, Shohei",665489,660271,strikeout_double_play,called_strike,...,NaN,1.36,0.35,0.35,30.1,NaN,NaN,NaN,NaN,NaN
952,SL,2025-10-29,89.7,1.90,6.53,"Snell, Blake",676391,605483,truncated_pa,ball,...,2.0,2.11,-0.12,0.12,53.6,NaN,NaN,NaN,NaN,NaN
5822,FF,2025-10-10,94.3,-2.49,5.44,"Flaherty, Jack",641487,656427,truncated_pa,ball,...,2.0,1.10,0.72,-0.72,27.1,NaN,NaN,NaN,NaN,NaN
6175,FF,2025-10-10,97.3,-1.94,5.56,"Kirby, George",656716,669923,truncated_pa,ball,...,NaN,1.21,0.71,-0.71,28.4,NaN,NaN,NaN,NaN,NaN
8469,SI,2025-10-07,95.2,-2.36,5.62,"Montero, Keider",668227,672456,truncated_pa,called_strike,...,1.0,1.62,1.30,1.30,38.1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761151,CH,2025-03-16,87.8,-1.99,5.65,"Fulmer, Carson",669224,608334,strikeout_double_play,swinging_strike,...,NaN,2.30,1.18,-1.18,NaN,18.215667,-9.930861,42.526982,32.256822,35.789567
761901,NaN,2025-03-16,NaN,NaN,NaN,"Kelly, Kevin",678391,687330,truncated_pa,ball,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
762497,SI,2025-03-16,93.6,-2.10,5.50,"Armstrong, Shawn",676113,542888,strikeout_double_play,swinging_strike,...,NaN,1.83,1.37,-1.37,NaN,NaN,NaN,NaN,NaN,NaN
765680,FC,2025-03-15,91.3,-2.39,5.58,"Vespi, Ben",624428,694978,truncated_pa,swinging_strike,...,NaN,1.60,-0.14,0.14,NaN,NaN,NaN,NaN,NaN,NaN


finding unique values

In [ ]:
unique_B = events2025['Play Description'].unique()
print(unique_B)

['Alejandro Kirk grounds into a double play, shortstop Mookie Betts to first baseman Freddie Freeman. Addison Barger out at 2nd. Alejandro Kirk out at 1st.'
 'Called Strike' 'Foul' ...
 'Jarred Kelenic flies out to center fielder Harrison Bader.'
 'Trevor Larnach grounds out, second baseman Ozzie Albies to first baseman Matt Olson.'
 'Ryan Jeffers doubles (1) on a line drive to left fielder Jake Marisnick.']


In [ ]:
grounds into double play